Import File

In [1]:
import pandas as pd
data = pd.read_csv('train.tsv',sep = '\t',header= None)
data

,0,1
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
...,...,...
5057054,do,N
5057055,daoine,S
5057056,beaga,N
5057057,.,N


In [2]:
data.columns = ['Token','mutation']
data['Token']

0           ansin
1               )
2              tá
3            níos
4              lú
            ...  
5057054        do
5057055    daoine
5057056     beaga
5057057         .
5057058       <S>
Name: Token, Length: 5057059, dtype: object

Tokenize pair of words 

In [3]:
list_add=[]
size= len(data)
for temp in range(size-1):
  list_add.append(str(data['Token'][temp])+' '+ str(data['Token'][temp+1]))
  temp+=1
list_add.insert(0,'ansin') 

In [4]:
list_add[1]

'ansin )'

In [5]:
tokens = {  'Bi-gram' : list_add,
            'Mutations': data['mutation']
    
}
tokens = pd.DataFrame(tokens)
tokens

,Bi-gram,Mutations
0,ansin,N
1,ansin ),N
2,) tá,N
3,tá níos,N
4,níos lú,N
...,...,...
5057054,mór do,N
5057055,do daoine,S
5057056,daoine beaga,N
5057057,beaga .,N


Cleaning dataset by removing unnecessary text

In [6]:
tokens['Bi-gram'] = tokens['Bi-gram'].str.replace("[<URI><USER><EMAIL><NUM>]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


#Naive-Bayes

---Accuracy is quite low due to a bug (Always predicting one class) 



In [11]:
tokens=tokens.iloc[:50000]
split_index=int(len(tokens)*0.3)
train=tokens.iloc[split_index:]
test=tokens.iloc[:split_index]
Y_train=train.iloc[:,:-1].values
X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
train


,Bi-gram,Mutations
15000,gach duine,N
15001,duine idir,N
15002,idir páistí,S
15003,páistí ',N
15004,' is,N
...,...,...
49995,agitato .,N
49996,.,N
49997,rinn,N
49998,rinn an,N


Calculating Prior Probabilities for each class

In [12]:
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    print(classes)
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

In [13]:
bow_H = {}
bow_N = {}
bow_T={}
bow_U={}
bow_S={}
count_H = 0
count_N = 0
count_T = 0
count_U = 0
count_S = 0

In [14]:
#tokens['Bi-gram'][1]
import re
ss=re.findall(r"[\w]+|[^\s\w]", tokens['Bi-gram'][1])

In [15]:
print(ss)

['ansin', ')']


Collecting bag of words(Unique words or Vocabulary) 

In [16]:
bog = {}
for line in tokens['Bi-gram']:
    for word in re.findall(r"[\w]+|[^\s\w]", line): #separating punctuation as tokens
        if word not in bog.keys():
            bog[word] = 1
        else:
            bog[word] += 1

In [17]:
len(bog)

11051

In [18]:
prior=calculate_prior(tokens,Y='Mutations')
print(prior)

['H', 'N', 'S', 'T', 'U']
[0.0079, 0.85888, 0.09976, 0.00314, 0.03032]


Collecting words in dictionaries which appeared in each class

In [19]:
#count which words appear in +ive tweet and -ive tweet
import re
for i in range(len(tokens['Bi-gram'])):
  for word in re.findall(r"[\w]+|[^\s\w]", tokens['Bi-gram'][i]):
      if tokens['Mutations'][i] == 'H':
          if word not in bow_H.keys():
              bow_H[word] = 1
              count_H += 1
          else:
              bow_H[word] += 1
              count_H += 1
      elif tokens['Mutations'][i] == 'U':
          if word not in bow_U.keys():
              bow_U[word] = 1
              count_U += 1
          else:
              bow_U[word] += 1
              count_U += 1
      elif tokens['Mutations'][i] == 'S':
          if word not in bow_S.keys():
              bow_S[word] = 1
              count_S += 1
          else:
              bow_S[word] += 1
              count_S += 1    
      elif tokens['Mutations'][i] == 'T':
          if word not in bow_T.keys():
              bow_T[word] = 1
              count_T += 1
          else:
              bow_T[word] += 1
              count_T += 1 
      else:
          if word not in bow_N.keys():
              bow_N[word] = 1
              count_N += 1
          else:
              bow_N[word] += 1
              count_N += 1                                                                                      

In [ ]:
pd.set_option('display.max_rows', None)
tokens['Bi-gram']
df = tokens[tokens['Mutations']=='H']
df

Sorting dictionaries by count

In [22]:
bow_N = {k: v for k, v in sorted(bow_N.items(), reverse = True, key=lambda item: item[1])}
bow_U = {k: v for k, v in sorted(bow_U.items(), reverse = True, key=lambda item: item[1])}
bow_S = {k: v for k, v in sorted(bow_S.items(), reverse = True, key=lambda item: item[1])}
bow_T = {k: v for k, v in sorted(bow_T.items(), reverse = True, key=lambda item: item[1])}
bow_H = {k: v for k, v in sorted(bow_H.items(), reverse = True, key=lambda item: item[1])}

In [23]:
len(bow_T)

103

In [24]:
print(count_H)
print(count_N)
print(count_S)
print(count_T)
print(count_U)

2000
164347
10569
318
3043


Calculating likelihood probabilities of each word and taking product with prior probabilities




In [25]:
import math
import numpy as np
from math import log
vocab_size=len(bog)
train_predictions = []
classes=5
y_pred=[]
for sentence in tokens['Bi-gram']:
    prob_H = 0
    prob_N = 0
    prob_T = 0
    prob_S = 0
    prob_U = 0
    for word in re.findall(r"[\w]+|[^\s\w]", sentence):
        try:
            prob_H += log((bow_H[word]+1)/(count_H+vocab_size))
        except:
            prob_H += log((0+1)/(count_H+vocab_size))
        try:
            prob_N += log((bow_N[word]+1)/(count_N+vocab_size))
        except:
            prob_N += log((0+1)/(count_N+vocab_size))
        try:
            prob_S += log((bow_S[word]+1)/(count_S+vocab_size))
        except:
            prob_S += log((0+1)/(count_S+vocab_size))    
        try:
            prob_T += log((bow_T[word]+1)/(count_T+vocab_size))
        except:
            prob_T += log((0+1)/(count_T+vocab_size))                        
        try:
            prob_U += log((bow_U[word]+1)/(count_U+vocab_size))
        except:
            prob_U += log((0+1)/(count_U+vocab_size)) 
    prob_dic={
        0:prob_H,
        1:prob_N,
        2:prob_S,
        3:prob_T,
        4:prob_U
    }
    
    post_prob=[1]*classes
    for x in range(len(prior)):
      post_prob[x]=prior[x]*prob_dic[x]
      
    y_pred.append(np.argmax(post_prob))  

In [26]:
import sys
np.set_printoptions(threshold=sys.maxsize)
#y_pred
post_prob

#np.argmax(lis)
#bow_T

[-0.11931443054953748,
 -12.273741995521846,
 -1.3334886946020867,
 -0.043197954462670475,
 -0.4188315077980591]

Performing label encoding by mapping each class with integer

In [27]:
labels=tokens['Mutations']

In [28]:
labels=labels.str.replace("H","0")
labels=labels.str.replace("N","1")
labels=labels.str.replace("S","2")
labels=labels.str.replace("T","3")
labels=labels.str.replace("U","4")
y_labels=[]
for x in labels:
  y_labels.append(int(x))
#y_labels

In [29]:
y_pred=np.array(y_pred)
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score
print(confusion_matrix(y_labels, y_pred))
print(accuracy_score(y_labels, y_pred))

[[    0     0     0   395     0]
 [   98     0     0 42846     0]
 [    0     0     0  4988     0]
 [    0     0     0   157     0]
 [    0     0     0  1516     0]]
0.00314


#Naive-Bayes 2nd Approach

---Tested it on small dataset(50k training instances) due to time constraint and acquired 45% accuracy


---suji04[Source code] https://github.com/Suji04/ML_from_Scratch




In [ ]:
#Calculate prior prob of each class 
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    #print(classes)
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior



In [ ]:
#To verify that the sum of all the prior class probabilities is equal to 1
res=calculate_prior(data,Y='mutation')
sum=0
for x in res:
  sum+=x
print(sum)

1.0


In [ ]:

def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    #print(df)
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    #print(p_x_given_y)
    return p_x_given_y


In [ ]:

def naive_bayes_categorical(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]
    #print(features)
    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        labels = sorted(list(df[Y].unique()))
        
        likelihood = [1]*len(labels)
        
        for j in range(len(labels)):
            for i in range(len(features)):

                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])
                
        # calculate posterior probability
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]
            
        Y_pred.append(np.argmax(post_prob))
        #print(Y_pred)

    return np.array(Y_pred) 

In [ ]:
from sklearn.model_selection import train_test_split
tokens=tokens.iloc[:500000]
train, test = train_test_split(tokens, test_size=.3, random_state=41)
Y_train=train.iloc[:,:-1].values
X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
print(len(train))
print(len(Y_train))
#print(type(train))
print(len(X_test))
print(len(Y_test))

350000
350000
150000
150000


In [ ]:
#Calling the Naive-Bayes
#This step with 50k total instances takes almost 6min 
import numpy as np
Y_pred = naive_bayes_categorical(train, X=X_test, Y="Mutations")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_test = le.fit_transform(Y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score
import sys
np.set_printoptions(threshold=sys.maxsize)
#print(Y_pred)
#print(Y_test)


In [ ]:
print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[ 129    0    0    0    0]
 [7101 5709   29    6   17]
 [ 787   18  680    0   12]
 [  14    9    0   15    0]
 [ 179   33   31    0  231]]
0.45093333333333335
